In [1]:
#
# NOTE: This machine learning program is for predicting TC formation, using
#       input dataset in the NETCDF format. The program treats different
#       2D input fields as different channels of an image. This specific
#       program requires a set of 12 2D-variables (12-channel image) and
#       consists of three stages
#       - Stage 1: reading NETCDF input and generating (X,y) data with a
#                  given image sizes, which are then saved by pickle;
#       - Stage 2: import the saved pickle (X,y) pair and build a CNN model
#                  with a given training/validation ratio, and then save
#                  the train model under tcg_CNN.model.
#       - Stage 3: import the trained model from Stage 2, and make a list
#                  of prediction from normalized test data.
#
# INPUT: This Stage 2 script requires two specific input datasets that are
#        generated from Step 1, including
#        1. tcg_X.pickle: data contains all images of yes/no TCG events, each
#           of these images must have 12 channels
#        2. tcg_y.pickle: data contains all labels of each image (i.e., yes
#           or no) of TCG corresponding to each data in X.
#
#        Remarks: Note that each channel must be normalized separealy. Also
#        the script requires a large memory allocation. So users need to have
#        GPU version to run this.
#
# OUTPUT: A CNN model built from Keras saved under tcg_CNN.model
#
# HIST: - 27, Oct 22: Created by CK
#       - 01, Nov 22: Modified to include more channels
#       - 17, Nov 23: cusomize it for jupiter notebook
#
# AUTH: Chanh Kieu (Indiana University, Bloomington. Email: ckieu@iu.edu)
#
#==========================================================================
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import pickle
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time

2023-02-20 16:38:53.882838: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
#
# read in data output from Part 1
#
pickle_in = open("tcg_X.pickle","rb")
X = pickle.load(pickle_in)
pickle_in = open("tcg_y.pickle","rb")
y = pickle.load(pickle_in)
y = np.array(y)
number_channels=X.shape[3]
print('Input shape of the X features data: ',X.shape)
print('Input shape of the y label data: ',y.shape)
print('Number of input channel extracted from X is: ',number_channels)

Input shape of the X features data:  (24, 30, 30, 12)
Input shape of the y label data:  (24,)
Number of input channel extracted from X is:  12


In [5]:
#
# test a range of loop for hyperparameter tuning. This is a test and have no impact to the rest of the
# code. 
#
dense_layers = [0, 5]
layer_sizes = [128, 256]
conv_layers = [3, 5]
for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            print(dense_layer, layer_size, conv_layer)

0 128 3
0 128 5
0 256 3
0 256 5
5 128 3
5 128 5
5 256 3
5 256 5


In [3]:
#
# build a range of CNN models with different number of dense layers, layer sizes
# convolution layers to optimize the performance
#
dense_layers = [0]
layer_sizes = [128]
conv_layers = [3]
for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print('--> Running configuration: ',NAME)

            model = Sequential()
            model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:],data_format="channels_last"))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())

            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            tensorboard = TensorBoard(log_dir="./logs/{}".format(NAME))

            model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                          metrics=['accuracy'])

            model.fit(X, y, batch_size=90, epochs=30, validation_split=0.1, callbacks=[tensorboard])
#
# save the model for the final step
#
model.save('tcg_CNN.model')

--> Running configuration:  3-conv-128-nodes-0-dense-1676929414


2023-02-20 16:43:36.084852: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-20 16:43:39.604912: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30987 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:89:00.0, compute capability: 7.0


Epoch 1/30


2023-02-20 16:43:46.777588: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401


1/1 [==============================] - 15s 15s/step - loss: 3230.0630 - accuracy: 0.4286 - val_loss: 31261.2871 - val_accuracy: 0.0000e+00
Epoch 2/30
1/1 [==============================] - 0s 35ms/step - loss: 13403.7676 - accuracy: 0.5714 - val_loss: 17684.6777 - val_accuracy: 0.0000e+00
Epoch 3/30
1/1 [==============================] - 0s 26ms/step - loss: 7580.3325 - accuracy: 0.5714 - val_loss: 132.5553 - val_accuracy: 0.0000e+00
Epoch 4/30
1/1 [==============================] - 0s 26ms/step - loss: 53.1010 - accuracy: 0.5714 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/30
1/1 [==============================] - 0s 26ms/step - loss: 8873.1826 - accuracy: 0.4286 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/30
1/1 [==============================] - 0s 27ms/step - loss: 8224.6494 - accuracy: 0.4286 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/30
1/1 [==============================] - 0s 26ms/step - loss: 4777.2402 - accuracy: 0.4286 - val_loss: 0.0000e+00 - v

INFO:tensorflow:Assets written to: tcg_CNN.model/assets


INFO:tensorflow:Assets written to: tcg_CNN.model/assets
